# Admin features for UCI

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import numpy as np
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from scipy.stats import zscore, mode
from itertools import product
pd.options.display.max_columns = None

In [ ]:
# Connect to DB
with open('G:/My Drive/RY_UCI/Research/LMS/uci_connect_string.txt', mode='r') as f:
    conn_str = f.read()
engine = create_engine(conn_str)

In [3]:
data_dir = Path('G:/My Drive/RY_UCI/Research/Mellon/Admin data')

In [4]:
# Codes for term name conversion
qtr_codes = {'03':'Winter', '14':'Spring', '51':'Summer', '92':'Fall'}
qtr_mos = {'Fall': 10, 'Winter': 1, 'Spring': 4, 'Summer': 7}

In [5]:
# Letter grade to grade point
letter_gp = {'A+': 4, 'A': 4, 'A-': 3.7, 'B+': 3.3, 'B': 3,
            'B-': 2.7, 'C+': 2.3, 'C': 2, 'C-': 1.7, 'D+': 1.3,
            'D': 1, 'D-': 0.7, 'F': 0}

In [6]:
COURSE_TYPES = ["LEC", "DIS", "SEM", "LAB", "others", "MISSING"]
# COURSE_COMBINED_TYPES contains the following:
COURSE_COMBINED_TYPES = [
    "-".join(c) for c in
    product(COURSE_TYPES, COURSE_TYPES)
    if c[0] < c[1]]
# MODALITIES = ["online", "inperson", "MISSING"]
CIP2_CATEGORIES = ["{:02}".format(i) for i in range(1, 62)] + ["MISSING", ]
COURSE_TYPES_COLS = 'units_type_' + pd.Index(COURSE_TYPES+COURSE_COMBINED_TYPES)
# MODALITIES_COLS = 'units_modality_' + pd.Index(MODALITIES)
CIP2_CATEGORIES_COLS = 'units_cip2_' + pd.Index(CIP2_CATEGORIES)

## Student-level features

In [ ]:
# Fetch student-level table from DB (with selected variables)
student_query = "SELECT mellon_id, birth_year, birth_month, gender, ethnicity, urm, citizenship_app, application_term_code, hs_gpa, sat_math_score, sat_verb_score, act_english_score, act_math_score FROM PRES.vw_student;"
st = pd.read_sql(student_query, engine)

In [7]:
# Read student-level table from local machine (with selected variables)
st = pd.read_csv(data_dir/'cleaned_student_background_data(20211222).csv',
                 usecols = ['mellon_id', 'birth_year', 'birth_month', 'gender', 'ethnicity', 'urm', 'citizenship_app', 'application_term_code', 'hs_gpa', 'sat_math_score', 'sat_verb_score', 'act_english_score', 'act_math_score'],
                 encoding='ISO-8859-1')

D:\Programs\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
# Recode variables for consistency
st['sex'] = st['gender'].map({'F': 'Female', 'M': 'Male', 'X': 'Other', 'U': 'NotIndicated'}).fillna('NotIndicated')
st['ethnicity'] = st['ethnicity'].map({
    'White non-Hispanic': 'White',
    'Asian / Asian American': 'Asian',
    'Hispanic': 'Hispanic',
    'Black': 'Black',
    'American Indian / Alaskan Native': 'Native Amr',
    'Pacific Islander': 'Hawaiian',
    'Two or more ethnicities':'2 or More',
    'Unknown / declined to state': 'Not Indic',
    'Unknown': 'Not Indic'    
}).fillna('Not Indic')
st['urm_status'] = np.where(
    st['citizenship_app'] == 'Not US Citizen',
    'International',
    np.where(
        st['urm'] == 1, 
        'Underrepresented Minority',
        'Non-Underrepresented Minority'
    )
)
st['hs_gpa'] = st['hs_gpa'].clip(0,5)
# st['modality'] = 'inperson'

In [9]:
# Rename variables for consistency
st = st.rename(columns={
    'hs_gpa': 'gpa_high_school',
    'sat_math_score': 'sat_math',
    'sat_verb_score': 'sat_verbal',
    'act_english_score': 'act_english',
    'act_math_score': 'act_math'
})

In [10]:
st.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134146 entries, 0 to 134145
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   mellon_id              134146 non-null  int64  
 1   birth_year             134082 non-null  float64
 2   birth_month            134101 non-null  float64
 3   gender                 133654 non-null  object 
 4   ethnicity              134146 non-null  object 
 5   urm                    114265 non-null  float64
 6   citizenship_app        62745 non-null   object 
 7   application_term_code  129865 non-null  float64
 8   gpa_high_school        90247 non-null   float64
 9   sat_math               64484 non-null   float64
 10  sat_verbal             35473 non-null   float64
 11  act_english            27815 non-null   float64
 12  act_math               27815 non-null   float64
 13  sex                    134146 non-null  object 
 14  urm_status             134146 non-nu

## Student-term features

In [ ]:
# Fetch student-term-level table from DB (with selected variables)
student_term_query = "SELECT mellon_id, term_code, current_units_completed_transfer, current_units_completed_total, gpa_cumulative, major_cip_code_1, major_cip_code_2 FROM PRES.vw_student_term;"
sttm = pd.read_sql(student_term_query, engine)

In [11]:
# Read student-term-level table from local machine (with selected variables)
sttm = pd.read_csv(data_dir/'cleaned_student_term_data(20211222).csv',
                   usecols = ['mellon_id', 'term_code', 'current_units_completed_transfer', 'current_units_completed_total', 'gpa_cumulative', 'major_cip_code_1', 'major_cip_code_2'],
                   encoding='ISO-8859-1')

In [12]:
# Modify data types
sttm['term_code'] = sttm['term_code'].astype(int)
sttm[['current_units_completed_transfer', 'current_units_completed_total']] = sttm[['current_units_completed_transfer', 'current_units_completed_total']].astype(float)
sttm[['major_cip_code_1', 'major_cip_code_2']] = sttm[['major_cip_code_1', 'major_cip_code_2']].replace({'&': np.nan}).astype(float)

In [13]:
# Merge students' birth info and compute age in each term
sttm = sttm.merge(st[['mellon_id', 'birth_year', 'birth_month']], on='mellon_id', how='left')
sttm['year'] = np.where(sttm['term_code'].notnull(), sttm['term_code'].astype(str).str[:4].astype(int), np.nan)
# sttm['term_start_date'] = pd.to_datetime(sttm['term_code'].apply(lambda x: f'{x[:4]}-{qtr_mos[qtr_codes[x[-2:]]]}-1' if x.isdigit() else np.nan), errors = 'coerce')
# sttm['birth_date'] = pd.to_datetime(sttm['birth_year'] +'-' + sttm['birth_month'] + '-1', errors = 'coerce')
# sttm['age'] = (sttm['term_start_date'] - sttm['birth_date']) / np.timedelta64(1, 'Y')
sttm['age'] = sttm['year'].subtract(sttm['birth_year'].astype(float).astype('Int64'))

In [14]:
# Recode variables for consistency
for i in np.arange(2)+1:
    sttm[f'cip2_major_{i}'] = sttm[f'major_cip_code_{i}'].apply(lambda x: '{:02}'.format(math.floor(x)) if pd.notnull(x) else 'MISSING')

In [15]:
def get_next_fall(term_code):
    if pd.isnull(term_code):
        return None
    elif term_code % 100 == 92:
        return (term_code + 100)
    else:
        return (100 * math.floor(term_code / 100) + 92)

def get_retention_list(term_codes):
    term_code_list = [int(term) for term in term_codes if pd.notnull(term)]
    return [get_next_fall(term) in term_code_list for term in term_codes]

In [16]:
# Compute retention flag: whether a given student in a given term enrolled in the following Fall
sttm['retention'] = sttm.groupby('mellon_id')['term_code'].transform(get_retention_list)

In [17]:
# Rename variables for consistency
sttm = sttm.rename(columns={
    'current_units_completed_total': 'units',
    'current_units_completed_transfer': 'units_transferred',
    'gpa_term': 'gpa_avg'
})

In [18]:
sttm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1105422 entries, 0 to 1105421
Data columns (total 14 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   mellon_id          1105422 non-null  int64  
 1   term_code          1105422 non-null  int32  
 2   units_transferred  1105422 non-null  float64
 3   units              1105422 non-null  float64
 4   gpa_cumulative     1105422 non-null  float64
 5   major_cip_code_1   775597 non-null   float64
 6   major_cip_code_2   0 non-null        float64
 7   birth_year         832685 non-null   float64
 8   birth_month        832686 non-null   float64
 9   year               1105422 non-null  float64
 10  age                832685 non-null   Float64
 11  cip2_major_1       1105422 non-null  object 
 12  cip2_major_2       1105422 non-null  object 
 13  retention          1105422 non-null  bool   
dtypes: Float64(1), bool(1), float64(8), int32(1), int64(1), object(2)
memory usage: 11

## Student-term-course features

In [19]:
def weightstats(x, weights=None, stat='mean'):
    '''
    Extended stats function that accommodates missing values and sample weights
    
    '''
    x = np.asarray(x)
    if weights is None:
        weights = np.ones(len(x))
    else:
        weights = np.asarray(weights).astype(float)
    indices = ((~np.isnan(x)) & (~np.isnan(weights)))
    if ~indices.any():
        return np.nan
    else:
        x = x[indices]
        weights = weights[indices]
        mean = np.dot(x.T, weights) / weights.sum()
        if stat == 'mean':
            return mean
        std = np.sqrt(np.dot(((x - mean) ** 2).T, weights) / weights.sum())
        if stat == 'std':
            return std

In [20]:
def agg_stcs(chunk):
    '''
    For a given chunk of student-course-level table, derive the aggregated student-term table
    
    '''
    # ID headers
    chunk_agg = chunk[['mellon_id', 'term_code']].drop_duplicates()
    print('ID headers done.')
    
    # Units by course type
    chunk['course_type'] = chunk['course_type'].fillna('MISSING').replace({'': 'MISSING'})
    chunk['course_type_rec'] = chunk['course_type'].where(
        chunk['course_type'].isin(['LEC', 'DIS', 'SEM', 'LAB', 'MISSING']),
        'others'
    )
    chunk['course_type_concat'] = chunk.groupby(['mellon_id', 'term_code', 'course_dept_code_and_num'], as_index=False)['course_type_rec'].transform(lambda x: x.sort_values().drop_duplicates().str.cat(sep='-')).replace({'': None})
    units_by_course_type = pd.pivot_table(chunk.groupby(['mellon_id', 'term_code', 'course_type_concat'], as_index=False)['units_completed'].sum(),
                                          index=['mellon_id', 'term_code'], columns='course_type_concat', values='units_completed',
                                          aggfunc='sum').add_prefix('units_type_')
    units_by_course_type = units_by_course_type.drop(columns=units_by_course_type.columns.difference(COURSE_TYPES_COLS))
    units_by_course_type[COURSE_TYPES_COLS.difference(units_by_course_type.columns)] = np.nan
    chunk_agg = chunk_agg.merge(units_by_course_type.reset_index(), how='left', on=['mellon_id', 'term_code'])
    print('Units by course type done.')
    
    # Units by course modality
#     chunk['meeting_location'] = chunk['meeting_location'].str.strip().str.upper().str.replace(r'(\t|\n|\s)+', '').replace({'': None})
#     chunk['format'] = np.where(
#         chunk['meeting_location'].str.contains('TBA') | chunk['meeting_location'].isnull(),
#         'MISSING',
#         np.where(
#             chunk['meeting_location'].str.contains(r'(ONLINE)|(REMOTE)'),
#             'online',
#             'inperson'
#         )
#     )
#     units_by_modality = pd.pivot_table(chunk.groupby(['mellon_id', 'term_code', 'format'], as_index=False)['units_completed'].sum(),
#                                        index=['mellon_id', 'term_code'], columns='format', values='units_completed',
#                                        aggfunc='sum').add_prefix('units_modality_')
#     units_by_modality = units_by_modality.drop(columns=units_by_modality.columns.difference(MODALITIES_COLS))
#     units_by_modality[MODALITIES_COLS.difference(units_by_modality.columns)] = np.nan
#     chunk_agg = chunk_agg.merge(units_by_modality.reset_index(), how='left', on=['mellon_id', 'term_code'])
#     print('Units by course modality done.')
    
    # Units by cip codes
    chunk['course_cip2'] = chunk.groupby(['term_code', 'course_code'])['cip2_major_1'].transform(lambda x: pd.Series.mode(x)[0] if len(pd.Series.mode(x)) > 0 else None).fillna('MISSING')
    units_by_cip2 = pd.pivot_table(chunk.groupby(['mellon_id', 'term_code', 'course_cip2'], as_index=False)['units_completed'].sum(),
                                   index=['mellon_id', 'term_code'], columns='course_cip2', values='units_completed',
                                   aggfunc='sum').add_prefix('units_cip2_')
    units_by_cip2 = units_by_cip2.drop(columns=units_by_cip2.columns.difference(CIP2_CATEGORIES_COLS))
    units_by_cip2[CIP2_CATEGORIES_COLS.difference(units_by_cip2.columns)] = np.nan   
    chunk_agg = chunk_agg.merge(units_by_cip2.reset_index(), how='left', on=['mellon_id', 'term_code'])
    print('Units by CIP codes done.')
    
    # Failed/incompleted/withdrawn units
    chunk_agg = chunk_agg.merge(
        chunk[chunk['final_grade']=='F'].groupby(['mellon_id', 'term_code'], as_index=False)['units_completed'].sum().rename(columns={'units_completed': 'units_failed'}),
        how='left', on=['mellon_id', 'term_code']
    )
    chunk_agg = chunk_agg.merge(
        chunk[chunk['final_grade']=='I'].groupby(['mellon_id', 'term_code'], as_index=False)['units_completed'].sum().rename(columns={'units_completed': 'units_incompleted'}),
        how='left', on=['mellon_id', 'term_code']
    )
    chunk_agg = chunk_agg.merge(
        chunk[chunk['final_grade']=='W'].groupby(['mellon_id', 'term_code'], as_index=False)['units_completed'].sum().rename(columns={'units_completed': 'units_withdrawn'}),
        how='left', on=['mellon_id', 'term_code']
    )
    print('Failed/incompleted/withdrawn units done.')
    
    # Fill 0s for all missing values so far
    chunk_agg = chunk_agg.fillna(0)
    
    # GPA stats
    chunk['grade_point'] = chunk['final_grade'].map(letter_gp)
    chunk['grade_point_z'] = chunk.groupby(['term_code', 'course_code'])['grade_point'].transform(zscore, nan_policy='omit')
    chunk_agg = chunk_agg.merge(
        chunk.groupby(['mellon_id', 'term_code']).apply(lambda x: weightstats(x['grade_point'], weights=x['units_completed'], stat='mean')).reset_index(name='gpa_avg'),
        how='left', on=['mellon_id', 'term_code']
    )
    chunk_agg = chunk_agg.merge(
        chunk.groupby(['mellon_id', 'term_code']).apply(lambda x: weightstats(x['grade_point'], weights=x['units_completed'], stat='std')).reset_index(name='gpa_stddev'),
        how='left', on=['mellon_id', 'term_code']
    )
    chunk_agg = chunk_agg.merge(
        chunk.groupby(['mellon_id', 'term_code']).apply(lambda x: weightstats(x['grade_point_z'], weights=x['units_completed'], stat='mean')).reset_index(name='gpa_zscore_avg'),
        how='left', on=['mellon_id', 'term_code']
    )
    chunk_agg = chunk_agg.merge(
        chunk.groupby(['mellon_id', 'term_code']).apply(lambda x: weightstats(x['grade_point_z'], weights=x['units_completed'], stat='std')).reset_index(name='gpa_zscore_stddev'),
        how='left', on=['mellon_id', 'term_code']
    )
    print('GPA stats done.')
    
    return chunk_agg

In [21]:
terms = [201292, 201392, 201492, 201592, 201692, 201792, 201892, 201992]

In [ ]:
# Fetch student-term-course-level table from DB (with selected variables), one term at a time
student_course_query = '''
SELECT mellon_id, term_code, course_code, course_dept_code_and_num, course_section_num, course_type, meeting_location, units_attempted, units_completed, final_grade 
FROM PRES.vw_stcs
WHERE term_code = %s;
'''
# Loop through all terms, and concatenate the aggregated table from each term
for term in terms:    
    chunk = pd.read_sql(student_course_query, engine, params=(term,))
    chunk = chunk.merge(sttm[['mellon_id', 'term_code', 'cip2_major_1']], how='left')
    chunk_agg = agg_stcs(chunk)
    try:
        stcs_agg = stcs_agg.append(chunk_agg)
    except:
        stcs_agg = chunk_agg
    print(f'Term {term} finished.')

In [22]:
# Read student-term-course-level table from local machine (with selected variables)
itr = pd.read_csv(data_dir/'cleaned_student_term_course_section(20211222).csv',
                  usecols = ['mellon_id', 'term_code', 'course_code', 'course_dept_code_and_num', 'course_section_num', 'course_type', 'meeting_location', 'units_attempted', 'units_completed', 'final_grade'],
                  encoding='ISO-8859-1', iterator=True, chunksize=1000000)
for chunk in itr:
    try:
        stcs = stcs.append(chunk[chunk['term_code'].isin(terms)])
    except:
        stcs = chunk[chunk['term_code'].isin(terms)]
stcs = stcs.merge(sttm[['mellon_id', 'term_code', 'cip2_major_1']], how='left')
stcs['cip2_major_1'] = stcs['cip2_major_1'].fillna('MISSING')

In [23]:
for term in terms:
    chunk_agg = agg_stcs(stcs[stcs['term_code']==term])
    try:
        stcs_agg = stcs_agg.append(chunk_agg)
    except:
        stcs_agg = chunk_agg
    print(f'Term {term} finished.')

ID headers done.


C:\Users\yuren\AppData\Local\Temp/ipykernel_21224/287612154.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['course_type'] = chunk['course_type'].fillna('MISSING').replace({'': 'MISSING'})
C:\Users\yuren\AppData\Local\Temp/ipykernel_21224/287612154.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['course_type_rec'] = chunk['course_type'].where(
C:\Users\yuren\AppData\Local\Temp/ipykernel_21224/287612154.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

Units by course type done.


C:\Users\yuren\AppData\Local\Temp/ipykernel_21224/287612154.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['course_cip2'] = chunk.groupby(['term_code', 'course_code'])['cip2_major_1'].transform(lambda x: pd.Series.mode(x)[0] if len(pd.Series.mode(x)) > 0 else None).fillna('MISSING')


Units by CIP codes done.
Failed/incompleted/withdrawn units done.


C:\Users\yuren\AppData\Local\Temp/ipykernel_21224/287612154.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['grade_point'] = chunk['final_grade'].map(letter_gp)
C:\Users\yuren\AppData\Local\Temp/ipykernel_21224/287612154.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['grade_point_z'] = chunk.groupby(['term_code', 'course_code'])['grade_point'].transform(zscore, nan_policy='omit')


GPA stats done.
Term 201292 finished.
ID headers done.
Units by course type done.
Units by CIP codes done.
Failed/incompleted/withdrawn units done.
GPA stats done.
Term 201392 finished.
ID headers done.
Units by course type done.
Units by CIP codes done.
Failed/incompleted/withdrawn units done.
GPA stats done.
Term 201492 finished.
ID headers done.
Units by course type done.
Units by CIP codes done.
Failed/incompleted/withdrawn units done.
GPA stats done.
Term 201592 finished.
ID headers done.
Units by course type done.
Units by CIP codes done.
Failed/incompleted/withdrawn units done.
GPA stats done.
Term 201692 finished.
ID headers done.
Units by course type done.
Units by CIP codes done.
Failed/incompleted/withdrawn units done.
GPA stats done.
Term 201792 finished.
ID headers done.
Units by course type done.
Units by CIP codes done.
Failed/incompleted/withdrawn units done.
GPA stats done.
Term 201892 finished.
ID headers done.
Units by course type done.
Units by CIP codes done.
Faile

C:\Users\yuren\AppData\Local\Temp/ipykernel_21224/3477483650.py:17: RuntimeWarning: invalid value encountered in double_scalars
  mean = np.dot(x.T, weights) / weights.sum()


GPA stats done.
Term 201992 finished.


In [24]:
stcs_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218917 entries, 0 to 36463
Data columns (total 92 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   mellon_id                  218917 non-null  int64  
 1   term_code                  218917 non-null  int64  
 2   units_type_DIS             218917 non-null  float64
 3   units_type_DIS-LAB         218917 non-null  float64
 4   units_type_DIS-LEC         218917 non-null  float64
 5   units_type_DIS-MISSING     218917 non-null  float64
 6   units_type_DIS-SEM         218917 non-null  float64
 7   units_type_DIS-others      218917 non-null  float64
 8   units_type_LAB             218917 non-null  float64
 9   units_type_LAB-LEC         218917 non-null  float64
 10  units_type_LAB-MISSING     218917 non-null  float64
 11  units_type_LEC             218917 non-null  float64
 12  units_type_LEC-MISSING     218917 non-null  float64
 13  units_type_LEC-others      218

## Combined dataset

In [25]:
combined = stcs_agg.merge(sttm.drop(columns=['birth_year', 'birth_month', 'major_cip_code_1', 'major_cip_code_2']), how='inner', on=['mellon_id', 'term_code'])
combined = combined.merge(st.drop(columns=['birth_year', 'birth_month', 'gender', 'citizenship_app', 'urm']), how='inner', on=['mellon_id'])

In [26]:
combined = combined[combined['application_term_code'] == combined['term_code']]

In [27]:
combined = combined.drop(columns=['mellon_id', 'term_code', 'application_term_code']).reset_index(drop=True)

In [28]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64892 entries, 0 to 64891
Columns: 106 entries, units_type_DIS to urm_status
dtypes: Float64(1), bool(1), float64(99), object(5)
memory usage: 52.1+ MB


In [29]:
combined.to_feather('G:/My Drive/RY_UCI/Research/Transfer Models/transfer_models/data/preprocessed/uci.feather')